In [59]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [60]:
import grader

# SQL Miniproject

## Introduction

The city of New York does restaurant inspections and assigns a grade. Inspections data for the last 4 years are available on s3 as an SQLite database, which you can import in the next few cells. These were extracted from a set of csv files and an xls file, as described in the <b>How we loaded the data</b> section


The raw data can be found [here](s3://dataincubator-course/coursedata/nyc_inspection_data.zip) and can be useful to look at. The file `RI_Webextract_BigApps_Latest.xls` contains a description of each of the datafiles and what the columns mean.

In [3]:
!aws s3 sync s3://dataincubator-course/coursedata/ . --exclude '*' --include 'nyc_inspection.db'

In [4]:
#This will load the pre-existing tables
%load_ext sql
%sql sqlite:///nyc_inspection.db

/opt/conda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


u'Connected: None@nyc_inspection.db'

To see what tables are in the database:

In [61]:
%%sql
SELECT * FROM sqlite_master WHERE "type"='table';

Done.


type,name,tbl_name,rootpage,sql
table,writer,writer,2,"CREATE TABLE writer (first_name, last_name, year)"
table,webextract,webextract,3,"CREATE TABLE webextract ( ""index"" BIGINT, ""CAMIS"" BIGINT, ""DBA"" TEXT, ""BORO"" BIGINT, ""BUILDING"" TEXT, ""STREET"" TEXT, ""ZIPCODE"" FLOAT, ""PHONE"" TEXT, ""CUISINECODE"" BIGINT, ""INSPDATE"" TEXT, ""ACTION"" TEXT, ""VIOLCODE"" TEXT, ""SCORE"" FLOAT, ""CURRENTGRADE"" TEXT, ""GRADEDATE"" TEXT, ""RECORDDATE"" TEXT)"
table,violations,violations,24393,"CREATE TABLE violations ( ""index"" BIGINT, ""STARTDATE"" TEXT, ""ENDDATE"" TEXT, ""CRITICALFLAG"" TEXT, ""VIOLATIONCODE"" TEXT, ""VIOLATIONDESC"" TEXT)"
table,cuisine,cuisine,24426,"CREATE TABLE cuisine ( ""index"" BIGINT, ""CUISINECODE"" BIGINT, ""CODEDESC"" TEXT)"
table,boroughs,boroughs,24428,"CREATE TABLE boroughs ( id INT, name TEXT)"
table,vio,vio,24429,"CREATE TABLE vio (CODE,INSPDATE, CAMIS, VIOLCODE,CUISINE)"
table,startdate,startdate,31072,"CREATE TABLE startdate (CAMIS,CUISINE,start_date)"
table,enddate,enddate,31344,"CREATE TABLE enddate (CAMIS,CUISINE,end_date)"
table,vio_by_cui,vio_by_cui,31616,"CREATE TABLE vio_by_cui (CUISINE,viocount)"
table,days_by_cui,days_by_cui,31617,"CREATE TABLE days_by_cui (CUISINE,CAMIS,DAYS)"


And to look at the format of an individual table (note that you may need to change types to get the answers in the right form):

In [62]:
%%sql
PRAGMA table_info(webextract) 

Done.


cid,name,type,notnull,dflt_value,pk
0,index,BIGINT,0,None,0
1,CAMIS,BIGINT,0,None,0
2,DBA,TEXT,0,None,0
3,BORO,BIGINT,0,None,0
4,BUILDING,TEXT,0,None,0
5,STREET,TEXT,0,None,0
6,ZIPCODE,FLOAT,0,None,0
7,PHONE,TEXT,0,None,0
8,CUISINECODE,BIGINT,0,None,0
9,INSPDATE,TEXT,0,None,0


## SQLite3

The project should be written in SQL. Between SQLite and PostgreSQL we recommend sqlite3 for this project.  You can use the sqlite command prompt by running this command in bash
```bash
sqlite3 cmd "DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"
```
Alternatively, you can run bash commands in a jupyter notebook by prepending the `!` in a code cell (notice that we conveniently get the output displayed

In [7]:
!sqlite3 cmd """\
DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"""

William|Shakespeare|1616
Francis|Fitzgerald|1896


Finally, we use the [ipython-sql extension](https://github.com/catherinedevlin/ipython-sql#ipython-sql) by first loaidng the sql extension and then running our code with the "magic" command in the first line
```python
%%sql sqlite://
```
Notice that the output table is formatted nicely as a nice HTML table.

This is our recommended technique.  However, the grader is expecting python objects and you may need to use list comprehensions to reformat this output

In [8]:
%%sql 
DROP TABLE IF EXISTS writer;
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);

SELECT * FROM writer;

Done.
Done.
1 rows affected.
1 rows affected.
Done.


first_name,last_name,year
William,Shakespeare,1616
Francis,Fitzgerald,1896


In [9]:
result = _
#This captures the output of the previous cell

In [10]:
result

first_name,last_name,year
William,Shakespeare,1616
Francis,Fitzgerald,1896


## How we loaded the data

For future reference, here is how you can load data in to SQL (with examples).  If you have a csv file you created with something like

```
!printf "Name,Age\nAlice,3\nBob,10" > sample.csv.nogit
```


Then Sqlite3 has a convenient [`.import` function](https://sqlite.org/cli.html#csv_import) which can create tables from `.csv` files.

```bash
sqlite> .import sample.csv.nogit sample
sqlite> SELECT * FROM sample;
```

The files may contain malformatted text.  Unfortunately, this is all too common.  As a stop gap, remember that [`iconv`](https://linux.die.net/man/1/iconv) is a unix utility that can convert files between different text encodings.

Alternatively, you can also read csv files using pandas and convert that into SQL via some sql magic (this is what we actually did).

```
import pandas as pd
sample = pd.read_csv('sample.csv.nogit')
%sql DROP TABLE IF EXISTS sample
%sql PERSIST sample
%sql SELECT * FROM sample;
```

## Question 1: null_entries

Return the number of inspections (CAMIS, INSPDATE pairs) that do not have a score. Remove the corresponding rows from the dataset for the rest of the questions in the assignment.

**Question:** How else might we have handled this?

In [12]:
%%sql 
SELECT COUNT(*) FROM (SELECT CAMIS,INSPDATE FROM  webextract WHERE SCORE IS NULL GROUP BY CAMIS,INSPDATE) 

Done.


COUNT(*)
28864


In [13]:
%%sql 
SELECT COUNT(*) FROM (SELECT CAMIS,INSPDATE FROM  webextract WHERE SCORE IS NOT NULL GROUP BY CAMIS,INSPDATE) 

Done.


COUNT(*)
160492


In [14]:
%%sql
SELECT COUNT(*) FROM (SELECT CAMIS,INSPDATE FROM webextract GROUP BY CAMIS,INSPDATE)


Done.


COUNT(*)
168747


In [212]:
abs(160492-168747)

8255

In [22]:
%%sql
DELETE FROM webextract WHERE SCORE IS NULL

33524 rows affected.


[]

In [6]:

def null_entries():
    return 8255

grader.score('sql__null_entries', null_entries)

Your score:  1


## Question 2: score_by_zipcode

Return a list of tuples of the form:

    (zipcode, mean score, number of restaurants)

for each of the 87 zipcodes in the city with over 100 restaurants. Use the score from the latest inspection date for each restaurant. Sort the list in ascending order by mean score.

**Note:** There is an interesting discussion here about what the mean score *means* in this dataset. Think about what we're actually calculating - does it represent what we're trying to understand about these zipcodes?

What if we use the average of a restaurant's inspections instead of the latest?

**Checkpoints:**
- Total unique restaurants: 24,361;
- Total restaurants in valid zipcodes: 19,172


In [374]:
%%sql
SELECT COUNT(*) FROM (SELECT CAMIS FROM webextract GROUP BY CAMIS)

Done.


COUNT(*)
25232


In [252]:
%%sql
SELECT COUNT(*),SUM(count) FROM 
(SELECT ZIPCODE,count FROM (SELECT ZIPCODE,COUNT(DISTINCT CAMIS) AS count FROM webextract GROUP BY ZIPCODE) WHERE (count>100))

Done.


COUNT(*),SUM(count)
87,19172


In [318]:
%%sql
SELECT ZIPCODE,avg,count FROM 
(SELECT ZIPCODE,COUNT(CAMIS) AS count, SUM(SCORE) AS score,SUM(SCORE)/COUNT(CAMIS) AS avg FROM
(SELECT ZIPCODE,CAMIS,INSPDATE,SCORE FROM 
(SELECT ZIPCODE,CAMIS,INSPDATE,SCORE FROM webextract GROUP BY ZIPCODE,
CAMIS,INSPDATE ORDER BY ZIPCODE,INSPDATE)
GROUP BY ZIPCODE,CAMIS) GROUP BY ZIPCODE)
WHERE (count>100) ORDER BY avg



Done.


ZIPCODE,avg,count
10451.0,9.14285714286,154
10001.0,9.14949494949,495
11430.0,9.68965517241,145
11236.0,9.83018867925,106
10472.0,10.1308411215,107
11234.0,10.1862068966,145
11217.0,10.3455284553,246
10306.0,10.5333333333,105
11368.0,10.5357142857,280
10462.0,10.5379310345,145


In [320]:
import sqlite3
con = sqlite3.connect('nyc_inspection.db')
cursor = con.cursor()
cursor.execute('SELECT * FROM sqlite_master WHERE "type"="table"')
cursor.execute('PRAGMA table_info(webextract)')
cursor.execute('DELETE FROM webextract WHERE SCORE IS NULL')
cursor.execute('SELECT ZIPCODE,avg,count FROM (SELECT ZIPCODE,COUNT(CAMIS) AS count, SUM(SCORE) AS score,SUM(SCORE)/COUNT(CAMIS) AS avg FROM (SELECT ZIPCODE,CAMIS,INSPDATE,SCORE FROM (SELECT ZIPCODE,CAMIS,INSPDATE,SCORE FROM webextract GROUP BY ZIPCODE,CAMIS,INSPDATE ORDER BY ZIPCODE,INSPDATE) GROUP BY ZIPCODE,CAMIS) GROUP BY ZIPCODE) WHERE (count>100) ORDER BY avg')
result = cursor.fetchall()
def score_by_zipcode():
    
    
    #print result
    output = []
    for item in result:
        new_item = (str(int(item[0])),float(item[1]),int(item[2]))
        output.append(new_item)
#print output
    return output

grader.score('sql__score_by_zipcode', score_by_zipcode)

Your score:  1.0


In [323]:
f = open('score_by_zip.csv','w')
for line in output:
    f.write(line[0]+',USA,'+str(line[1])+'\n')
f.close()

## Question 3: score_by_map

The above are not terribly enlightening.  Use [CartoDB](http://cartodb.com/) to produce a map of average scores by zip code.  You can sign up for a free trial.

You will have to use their wizard to plot the data by [zipcode](https://carto.com/learn/guides/analysis/georeference). You will need to specify "USA" in the countryfield.  Then use the "share" button to return a link of the form [https://x.cartodb.com/](https://x.cartodb.com/).

**For fun:** How do JFK, Brighton Beach, Liberty Island (home of the Statue of Liberty), Financial District, Chinatown, and Coney Island fare?

**For more fun:** Plot restaurants as pins on the map, allowing the user to filter by "low", "middling", or "high"-scoring restaurants. You can use a CASE WHEN statement to create the different groups based on score thresholds.

In [326]:
def score_by_map():
    # must be url of the form https://x.cartodb.com/...
    return "https://rayding95.carto.com/builder/873b5b42-b129-46f7-8bb7-99f4798902a7/embed"

grader.score('sql__score_by_map', score_by_map)

Your score:  1.0


## Question 4: score_by_borough
Return a list of tuples of the form:

    (borough, mean score, number of restaurants)

for each of the city's five boroughs. Sort the list in ascending order by grade.

**Hint:** You will have to perform a join with the `boroughs` table. The borough names should be reported in ALL CAPS.

**Checkpoint:**
- Total restaurants in valid boroughs: 24,350

In [335]:
%%sql
SELECT COUNT(*) FROM (SELECT CAMIS,BORO,SCORE FROM webextract WHERE BORO IS NOT NULL AND BORO IS NOT 0 GROUP BY CAMIS)

Done.


COUNT(*)
24350


In [352]:
%%sql
SELECT boroughs.name,avg,count FROM 
(SELECT BORO,COUNT(CAMIS) AS count, SUM(SCORE) AS score,SUM(SCORE)/COUNT(CAMIS) AS avg FROM
(SELECT BORO,CAMIS,INSPDATE,SCORE FROM 
(SELECT BORO,CAMIS,INSPDATE,SCORE FROM webextract WHERE BORO IS NOT NULL AND BORO IS NOT 0 GROUP BY BORO,
CAMIS,INSPDATE ORDER BY BORO,INSPDATE)
GROUP BY BORO,CAMIS) GROUP BY BORO)
OUTTER JOIN boroughs ON BORO=boroughs.id
ORDER BY avg 

Done.


name,avg,count
THE BRONX,10.7768522578,2281
MANHATTAN,11.4896370438,9891
BROOKLYN,11.5968541631,5849
QUEENS,11.8225955326,5417
STATEN ISLAND,11.899122807,912


In [354]:

cursor.execute('SELECT boroughs.name,avg,count FROM (SELECT BORO,COUNT(CAMIS) AS count, SUM(SCORE) AS score,SUM(SCORE)/COUNT(CAMIS) AS avg FROM(SELECT BORO,CAMIS,INSPDATE,SCORE FROM (SELECT BORO,CAMIS,INSPDATE,SCORE FROM webextract WHERE BORO IS NOT NULL AND BORO IS NOT 0 GROUP BY BORO,CAMIS,INSPDATE ORDER BY BORO,INSPDATE) GROUP BY BORO,CAMIS) GROUP BY BORO) OUTTER JOIN boroughs ON BORO=boroughs.id ORDER BY avg')
result1 = cursor.fetchall()


[(u'THE BRONX', 10.776852257781675, 2281), (u'MANHATTAN', 11.48963704377717, 9891), (u'BROOKLYN', 11.596854163104805, 5849), (u'QUEENS', 11.82259553258261, 5417), (u'STATEN ISLAND', 11.899122807017545, 912)]


In [357]:
def score_by_borough():
    output = []
    for item in result1:
        new_item = (str(item[0]),float(item[1]),int(item[2]))
        output.append(new_item)
    return output

grader.score('sql__score_by_borough', score_by_borough)

Your score:  1.0


## Question 5: violations_by_cuisine

We want to look at violations themselves now.  We'll need to think more carefully about what we're measuring, since most restaurants have many inspections with possibly multiple violations per inspection, or long stretches of inspections with no violations.

There are many ways to deal with this normalization issue, but we'll go with a fairly straightforward one: dividing the number of violations by the length of time (in years) the restaurant has been open.  As a proxy for the length, we'll look at the difference between the oldest and newest inspection date, treating anything less than 30 days as 30 days (to account for those that were only inspected once, we'll assume everything was open for at least a month).

Since there are so many restaurants, we'll group them by cuisine and do a weighted average by computing 

    (total violations for a cuisine) / (total restaurant-years for that cuisine)

Return a list of 75 tuples of the form

    (cuisine name, reports per restaurant-year)
    
for cuisines with at least 100 violations total, ordered by increasing number of reports per restaurant-year
    
**Note:** This isn't the only way to normalize things.  How would other ways affect the computation?  If you similarly wanted to compute an average score by cuisine, how might you go about doing that?
    
**Checkpoint:**
- Total entries from valid cuisines: 522,410

In [63]:
%%sql

DROP TABLE IF EXISTS vio;
CREATE TEMP TABLE IF NOT EXISTS vio (CODE,INSPDATE, CAMIS, VIOLCODE,CUISINE);
INSERT INTO vio 
SELECT webextract.CUISINECODE AS CODE,webextract.INSPDATE AS INSPDATE,webextract.CAMIS AS CAMIS,webextract.VIOLCODE AS VIOCODE,cuisine.CODEDESC AS CUISINE 
FROM webextract INNER JOIN cuisine ON webextract.CUISINECODE=cuisine.CUISINECODE ORDER BY CAMIS,INSPDATE

Done.
Done.
531935 rows affected.


[]

In [64]:
%%sql

DROP TABLE IF EXISTS startdate;
CREATE TEMP TABLE IF NOT EXISTS startdate (CAMIS,CUISINE,start_date);
INSERT INTO startdate
SELECT CAMIS,CUISINE,INSPDATE AS start_date FROM (SELECT * FROM vio ORDER BY CAMIS,INSPDATE DESC) GROUP BY CUISINE,CAMIS

Done.
Done.
25232 rows affected.


[]

In [65]:
%%sql
DROP TABLE IF EXISTS enddate;
CREATE TEMP TABLE IF NOT EXISTS enddate (CAMIS,CUISINE,end_date);
INSERT INTO enddate
SELECT CAMIS,CUISINE,INSPDATE AS end_date FROM vio GROUP BY CUISINE,CAMIS

Done.
Done.
25232 rows affected.


[]

In [74]:
%%sql
DROP TABLE IF EXISTS vio_by_cui;
CREATE TABLE IF NOT EXISTS vio_by_cui (CUISINE,viocount);
INSERT INTO vio_by_cui 
SELECT CUISINE,viocount FROM 
(SELECT CUISINE,COUNT(CAMIS) AS totalcount,COUNT(VIOLCODE) AS viocount FROM 
(SELECT INSPDATE,CAMIS,VIOLCODE,CUISINE
FROM vio GROUP BY INSPDATE,CAMIS,VIOLCODE) GROUP BY CUISINE) WHERE viocount>=100

Done.
Done.
75 rows affected.


[]

In [71]:
%%sql
DROP TABLE IF EXISTS days_by_cui;
CREATE TABLE IF NOT EXISTS days_by_cui (CUISINE,CAMIS,DAYS);
INSERT INTO days_by_cui
SELECT CUISINE,CAMIS,(julianday(end_date)-julianday(start_date)) AS days FROM 
(SELECT startdate.CAMIS,startdate.CUISINE,startdate.start_date,enddate.end_date FROM startdate 
 INNER JOIN enddate ON startdate.CAMIS=enddate.CAMIS AND startdate.CUISINE=enddate.CUISINE);
UPDATE days_by_cui 
SET DAYS=30 WHERE DAYS<30




Done.
Done.
25232 rows affected.
3067 rows affected.


[]

In [72]:
%%sql

SELECT vio_by_cui.CUISINE,SUM(DAYS) AS TOTAL,vio_by_cui.viocount FROM days_by_cui  
INNER JOIN vio_by_cui ON days_by_cui.CUISINE=vio_by_cui.CUISINE GROUP BY days_by_cui.CUISINE

Done.


CUISINE,TOTAL,viocount
Afghan,8857.0,300
African,51896.0,1843
American,4735376.47431,125453
Armenian,22813.0,632
Asian,204250.0,7652
Australian,8313.0,248
Bagels/Pretzels,131133.0,3657
Bakery,538990.0,16095
Bangladeshi,20875.0,803
Barbecue,33093.0,799


In [75]:
import sqlite3
con_new = sqlite3.connect('nyc_inspection.db')
cursor1 = con_new.cursor()
cursor1.execute('SELECT vio_by_cui.CUISINE,SUM(DAYS) AS TOTAL,vio_by_cui.viocount FROM days_by_cui INNER JOIN vio_by_cui ON days_by_cui.CUISINE=vio_by_cui.CUISINE GROUP BY days_by_cui.CUISINE')
result5 = cursor1.fetchall()
output = []
for cui in result5:
    c = cui[2]
    cuisine = cui[0]
    r_y = cui[1]/365.0
    avg = c*1.0/r_y
    output.append((cuisine,avg))
print output

[(u'Afghan', 12.36310263068759), (u'African', 12.962367041775858), (u'American ', 9.669842566575483), (u'Armenian', 10.111778371980888), (u'Asian', 13.674320685434518), (u'Australian', 10.888969084566341), (u'Bagels/Pretzels', 10.179016723479215), (u'Bakery', 10.899413718250802), (u'Bangladeshi', 14.040479041916168), (u'Barbecue', 8.812588764995619), (u'Bottled beverages, including water, sodas, juices, etc.', 7.013550390514726), (u'Brazilian', 12.83659650412052), (u'Caf\xe9/Coffee/Tea', 6.713791586509789), (u'Cajun', 9.234628975265018), (u'Caribbean', 11.978653881825835), (u'Chicken', 10.065290453587084), (u'Chinese', 13.311110046155827), (u'Chinese/Cuban', 14.92717086834734), (u'Chinese/Japanese', 15.591233936532914), (u'Continental', 10.110888801217381), (u'Creole', 15.90112921852945), (u'Czech', 10.094729965156795), (u'Delicatessen', 12.667556925927533), (u'Donuts', 6.073439018207933), (u'Eastern European', 12.204188946329126), (u'Egyptian', 10.106619987269255), (u'English', 9.2686

In [76]:
def score_by_cuisine():
    return output

grader.score('sql__score_by_cuisine', score_by_cuisine)

Your score:  0.96


## Question 6: specific_violations_by_cuisine
Which cuisines tend to have a disproportionate number of what which violations? Answering this question isn't easy becuase you have to think carefully about normalizations.

1. More popular cuisine categories will tend to have more violations just becuase they represent more restaurants.
2. Similarly, some violations are more common.  For example, knowing that "Equipment not easily movable or sealed to floor" is a common violation for Chinese restuarants is not particularly helpful when it is a common violation for all restaurants.

The right quantity is to look at is the conditional probability of a specific type of violation given a specific cuisine type and divide it by the unconditional probability of the violation for the entire population. Taking this ratio gives the right answer.  Return the 20 highest ratios of the form:

    ((cuisine, violation), ratio, count)

**Hint:**
1. You might want to check out this [Stackoverflow post](http://stackoverflow.com/questions/972877/calculate-frequency-using-sql).
2. The definition of a violation changes with time.  For example, 10A can mean two different things "Toilet facility not maintained ..." or "Vermin or other live animal present ..." when things were prior to 2003. To deal with this, you should limit your analysis to violation codes with end date after Jan 1, 2014. (This end date refers to the validity time ranges in Violation.txt).
3. The ratios don't mean much when the number of violations of a given type and for a specific category are not large (why not?).  Be sure to filter these out.  We chose 100 as our cutoff.

**Checkpoint:**
- Top 20 ratios mean: 2.360652529900757

In [77]:
%%sql
DROP TABLE IF EXISTS act_vio;
CREATE TEMP TABLE IF NOT EXISTS act_vio (ENDDATE,VIOLCODE,VIOLATION);
INSERT INTO act_vio
SELECT ENDDATE,VIOLATIONCODE AS VIOLCODE,VIOLATIONDESC FROM
(SELECT ENDDATE,VIOLATIONCODE,VIOLATIONDESC FROM violations GROUP BY VIOLATIONCODE) WHERE ENDDATE>'2014-01-01'

Done.
Done.
112 rows affected.


[]

In [78]:
%%sql
DROP TABLE IF EXISTS vio_counts;
CREATE TEMP TABLE IF NOT EXISTS vio_counts (VIOLCODE,CUISINE,spec_count);
INSERT INTO vio_counts

SELECT VIOLCODE,CUISINE,COUNT(VIOLATION) AS spec_count FROM
(SELECT vio.CUISINE,vio.VIOLCODE, act_vio.VIOLATION FROM vio INNER JOIN act_vio ON vio.VIOLCODE=act_vio.VIOLCODE) 
GROUP BY CUISINE,VIOLCODE

Done.
Done.
4436 rows affected.


[]

In [79]:
%%sql
DROP TABLE IF EXISTS vio_counts_code;
CREATE TEMP TABLE IF NOT EXISTS vio_counts_code (VIOLCODE,count);
INSERT INTO vio_counts_code
SELECT VIOLCODE,SUM(spec_count) AS count FROM vio_counts GROUP BY VIOLCODE;
DROP TABLE IF EXISTS vio_counts_cui;
CREATE TEMP TABLE IF NOT EXISTS vio_counts_cui (CUISINE,count);
INSERT INTO vio_counts_cui
SELECT CUISINE,SUM(spec_count) AS count FROM vio_counts GROUP BY CUISINE;


Done.
Done.
93 rows affected.
Done.
Done.
84 rows affected.


[]

In [81]:
%%sql
DROP TABLE IF EXISTS results;
CREATE TABLE IF NOT EXISTS results (CUISINE,VIOLATION,ratio,count);
INSERT INTO results
SELECT CUISINE,VIOLATION,cond_prob/uncond_prob AS ratio,spec_count FROM
(SELECT CUISINE,VIOLATION,spec_count,spec_count*1.0/cui_count AS cond_prob,code_count*1.0/522410 AS uncond_prob FROM
(SELECT vio_counts.CUISINE,act_vio.VIOLATION,vio_counts.spec_count,vio_counts_cui.count AS cui_count,vio_counts_code.count AS code_count
FROM vio_counts INNER JOIN vio_counts_cui ON vio_counts_cui.CUISINE=vio_counts.CUISINE
INNER JOIN vio_counts_code ON vio_counts_code.VIOLCODE=vio_counts.VIOLCODE 
INNER JOIN act_vio ON vio_counts.VIOLCODE=act_vio.VIOLCODE
WHERE vio_counts.spec_count>=100)) ORDER BY ratio DESC


Done.
Done.
753 rows affected.


[]

In [82]:
import sqlite3
con2 = sqlite3.connect('nyc_inspection.db')
cursor2 = con2.cursor()
cursor2.execute('SELECT * FROM results LIMIT 20')
result6 = cursor2.fetchall()
output = []
avg = 0
for cui in result6:
    c = int(cui[3])
    cuisine = cui[0]
    violation = cui[1]
    ratio = float(cui[2])
    avg+=ratio
    output.append(((cuisine,violation),ratio,c))
print avg/20

2.46310395834


In [83]:
def violation_by_cuisine():
    return output

grader.score('sql__violation_by_cuisine', violation_by_cuisine)

Your score:  0.925


*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*